In [17]:
import pandas as pd

df_transformer = pd.read_csv('/home/sanjana/factual_evaluation_source_based/datasets/sota_annotations/dialogue_aggrefact.csv')

len(df_transformer)

1550

In [18]:
df_transformer.head()[:1]

,Unnamed: 0,DocID,Dialogue,Model,Summary,w/ Error,CorefE,CorefE_text,CircE,CircE_text,...,GramE_text,PredE,PredE_text,SubjObjE,SubjObjE_text,OtherE,OtherE_text,LinkE,LinkE_text,origin
0,0,13809941,Thelma: i dont have anything to wear\nLouisa: ...,human_ref,Louisa will lend Thelma her red velvet dress.,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval


In [62]:
import psycopg2

In [63]:
conn = psycopg2.connect(
   database="d3mhp3r7jhume9", 
    user='sgqokpuggkrkve', 
    password='ff59f8ca03f950cef2e98fb4ae4ff44bdf7136ff37da62215672902c613297d6', 
    host='ec2-18-211-172-50.compute-1.amazonaws.com', 
    port = '5432'
    
)

In [64]:
conn.autocommit = True

# #Creating a cursor object using the cursor() method
# cursor = conn.cursor()

# #Preparing query to create a database
# sql = '''CREATE database gpt4_summaries_short_qualifying''';

# #Creating a database
# cursor.execute(sql)
# print("Database created successfully........")

# #Closing the connection
# conn.close()

In [65]:
cursor = conn.cursor()

sql_drop = '''DROP TABLE IF EXISTS generated_summaries;'''
#Preparing query to create a database
sql = '''CREATE TABLE generated_summaries (
    uuid uuid DEFAULT gen_random_uuid() PRIMARY KEY, 
    docid TEXT NOT NULL ,
    model TEXT NOT NULL, 
    summary TEXT NOT NULL, 
    dialogue TEXT
);'''

#Creating a database
cursor.execute(sql_drop)
cursor.execute(sql)
print("Table created successfully........")
conn.commit()

Table created successfully........


In [66]:
sql_drop = '''DROP TABLE IF EXISTS label;'''

create_table_label_str = '''CREATE TABLE label (
    uuid uuid DEFAULT gen_random_uuid() PRIMARY KEY, 
    user_id TEXT NOT NULL,
    docid TEXT NOT NULL,
    model TEXT NOT NULL,
    nonfactual_spans TEXT ,
    evidence TEXT ,
    error_type TEXT,
    summary TEXT NOT NULL,
    dialogue TEXT
);'''

cursor.execute(sql_drop)
cursor.execute(create_table_label_str)
print("Table created successfully........")
conn.commit()

Table created successfully........


In [67]:
# df_path = '/home/sanjana/explainable_factual_evaluation/datasets/short_dialogue/model_generated/gpt4_generated_summaries/RefMatters_DialogSum_test.csv'
df_path = '/home/sanjana/factual_evaluation_source_based/annotations/round1_aggrefact/gpt4_summaries_short.csv'
# df_path = '/home/sanjana/factual_evaluation_source_based/annotations/sample/sample_annotations_sanjana.csv'



In [68]:
df = pd.read_csv(df_path)
# df_errors_intrinsic = df[df['error_type'] == 'Intrinsic_Error']
# df_errors_intrinsic_sample = df_errors_intrinsic.sample(4)
# df_errors_intrinsic_sample

In [69]:
# from random import shuffle
# df_sample = pd.concat([df_errors_intrinsic_sample, df_errors_extrinsic_sample, df_noerrors])
# # shuffle(df_sample)
# df_sample = df_sample.sample(frac = 1)
# df_sample

In [70]:
import pandas as pd 
import re
# df = pd.read_csv(df_path)
print(len(df))
cursor.execute("""SELECT * from generated_summaries""")
print('Size before adding' , len(cursor.fetchall()))
for idx, row in df.iterrows():
    # print(row)
    docid = row['DocID']
    dialogue = row['Dialogue']
    model = row['Model']
    origin = row['origin']
    # origin = 'sample'
    summary = row['Summary']
    if 'Summary' in df.keys(): 
        # and row['DocID'] in filter_ids:
        generic_summary = row['Summary']
        summary_uuid_generic = f'{docid}_{model}_gen'
        summary = re.sub( "'", r"''", summary)
        dialogue = re.sub( "'", r"''", dialogue)
        value_str = f"""'{docid}', '{model}', '{summary}', '{dialogue}'"""
        # print(re.findall(r"\'(\w+)\'", dialogue))
        cursor.execute(f"""INSERT INTO generated_summaries(docid, model, summary, dialogue) VALUES ({value_str})""",
                                                        )
        

# conn.close()

325
Size before adding 0


In [73]:
cursor.execute('''SELECT * from label''')
result = cursor.fetchall();
result
conn.commit()

In [74]:
result

[('b6273d21-9a0c-4033-8e2b-908d0a98c143',
  'sanjana',
  '13611370',
  'gpt4-32k-0613',
  None,
  None,
  None,
  "Derek asks Judy if she can feed his animals on Friday and Saturday as she isn''t leaving for the weekend. She agrees and Derek plans to give her his keys on Thursday. He also promises to provide her with more details later.",
  'Derek: Judy, r you leaving for the weekend?\r\nJudy: Nah\r\nDerek: So can you feed my animals on Friday and Saturday?\r\nJudy: sure, no problem\r\nDerek: Thank you :) Maybe on Thursday I would give you my keys?\r\nJudy: Ok\r\nDerek: Thanks :) Later I will tell you the details :)\r\nJudy: surely')]